# Install Detectron 2 and its dependencies 

In [ ]:
!pip install pyyaml==5.1
!pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html



# Imports

In [ ]:
import torch
assert torch.__version__.startswith("1.8") 
import torchvision
import cv2
from google.colab import drive

#Import some common imports
import itertools
import os
import numpy as np
import json
import random
import copy
import matplotlib.pyplot as plt
%matplotlib inline

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.utils.visualizer import ColorMode
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog
from detectron2.data import detection_utils as utils
import detectron2.data.transforms as T
from detectron2.engine import DefaultTrainer
from detectron2.data import build_detection_test_loader, build_detection_train_loader
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# Prepare the dataset

## Mount to drive

In [ ]:
drive.mount('/content/drive') #This step should be avoided if data is not placed on drive

Note that the following code has taken inspiration from different resources: 
The function to convert data to coco format has taken inspiration from the "Detectron 2 compare models + augmentation" kaggle. URL:https://www.kaggle.com/code/dhiiyaur/detectron-2-compare-models-augmentation

## Make function to convert data to COCO format

In [ ]:
def get_data_dicts(directory, classes):
    dataset_dicts = []
    for filename in [file for file in os.listdir(directory) if file.endswith('.json')]:
        json_file = os.path.join(directory, filename)
        with open(json_file) as f:
            img_anns = json.load(f)

        record = {}
        
        filename = os.path.join(directory, img_anns["imagePath"])
        
        record["file_name"] = filename
        record["image_id"] = filename
        record["height"] = 480
        record["width"] = 640
      
        annos = img_anns["shapes"]
        objs = []
        for anno in annos:
            px = [a[0] for a in anno['points']] # x coord
            py = [a[1] for a in anno['points']] # y-coord
            poly = [(x, y) for x, y in zip(px, py)] # poly 
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": classes.index(anno['label']),
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

## Load data

In [ ]:
#List the five classes 
classes = ['sink', 'door', 'bed', 'screen', 'socket']

#Give path to data - if path is not on drive change the path before running
data_path = '/content/drive/MyDrive/robotdata/Blue_ocean_dataset1/'

for d in ["train", "valid", "test"]:
    DatasetCatalog.register(
        "robot_" + d, 
        lambda d=d: get_data_dicts(data_path+d, classes) #uses get_data_dicts function to make data into COCO format
    )
    MetadataCatalog.get("robot_" + d).set(thing_classes=classes)

#Collect the data into metadatacatalog 
microcontroller_metadata = MetadataCatalog.get("robot_train")

#Convert trainingset to COCO format and save as train_dicts
train_dicts = get_data_dicts(data_path+'train', classes)

#Convert validationset to COCO format and save as valid_dicts
valid_dicts = get_data_dicts(data_path+'valid', classes)

#Convert testset to COCO format and save as test_dicts
test_dicts = get_data_dicts(data_path+'test', classes)


### Visualize loaded data

In [ ]:
#Visualize for checking the data is loaded properly 
for d in random.sample(train_dicts, 5):
    img = cv2.imread(d["file_name"])
    v = Visualizer(img[:, :, ::-1], metadata=microcontroller_metadata, scale=0.5)
    v = v.draw_dataset_dict(d)
    plt.figure(figsize = (14, 10))
    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    plt.show()

# Testing backbone 

## C4 backbone

### Train on baseline model

In [ ]:
cfg_C4 = get_cfg()
cfg_C4.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_C4_1x.yaml")) #Load the config file
cfg_C4.DATASETS.TRAIN = ('robot_train',)
cfg_C4.DATASETS.TEST = ('robot_valid',)   
cfg_C4.DATALOADER.NUM_WORKERS = 2
cfg_C4.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_C4_1x.yaml") #Load the weights
cfg_C4.SOLVER.IMS_PER_BATCH = 4 #put to 4 or else it will not run on collab due to limited resources
cfg_C4.SOLVER.MAX_ITER = 1000
cfg_C4.SOLVER.STEPS = ()
cfg_C4.MODEL.ROI_HEADS.NUM_CLASSES = 5 #number of classes

os.makedirs(cfg_C4.OUTPUT_DIR, exist_ok=True)#make directory for output files from model
trainer_C4 = DefaultTrainer(cfg_C4)  #Use default trainer
trainer_C4.resume_or_load(resume=False)
trainer_C4.train() #train model

#### Tensorboard over training

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

### Test and evaluate model

In [ ]:
cfg_C4.MODEL.WEIGHTS = os.path.join(cfg_C4.OUTPUT_DIR, "model_final.pth")
cfg_C4.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8 #set threshold
cfg_C4.DATASETS.TEST = ("robot_test",) # select testset 
predictor = DefaultPredictor(cfg_C4)

#### Evaluation matrix

In [ ]:
evaluator = COCOEvaluator("robot_test", output_dir="./output") #Use COCO evaluator
val_loader = build_detection_test_loader(cfg_C4, "robot_test")
print(inference_on_dataset(predictor.model, val_loader, evaluator)) #print evaluation matrix

#### Visualization of predictions

In [ ]:

for d in random.sample(test_dicts, 5): #gives 5 random samples from the testset
    im = cv2.imread(d["file_name"])
    outputs = predictor(im) 
    v = Visualizer(im[:, :, ::-1],
                   metadata=microcontroller_metadata, 
                   scale=0.5
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize=(14, 10))
    plt.imshow(out.get_image()[:, :, ::-1][..., ::-1])

### Save model

In [ ]:
torch.save(cfg_C4, 'model_final.pth')

## DC5

### Train on baseline model

In [ ]:
cfg_dc5 = get_cfg()
cfg_dc5.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_DC5_1x.yaml")) #Load model 
cfg_dc5.DATASETS.TRAIN = ('robot_train',)
cfg_dc5.DATASETS.TEST = ('robot_valid',)  
cfg_dc5.DATALOADER.NUM_WORKERS = 2
cfg_dc5.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_DC5_1x.yaml") #Load weights from model
cfg_dc5.SOLVER.IMS_PER_BATCH = 4 #set to 4 or collab will not run due to limited resources
cfg_dc5.SOLVER.MAX_ITER = 1000
cfg_dc5.SOLVER.STEPS = ()
cfg_dc5.MODEL.ROI_HEADS.NUM_CLASSES = 5 #Number of classes

os.makedirs(cfg_dc5.OUTPUT_DIR, exist_ok=True) #Create directory for output
trainer_dc5 = DefaultTrainer(cfg_dc5)  #Use default trainer
trainer_dc5.resume_or_load(resume=False)
trainer_dc5.train() #Train model 

#### Tensorboard over training

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

### Test and evaluation

In [ ]:
cfg_dc5.MODEL.WEIGHTS = os.path.join(cfg_dc5.OUTPUT_DIR, "model_final.pth")
cfg_dc5.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8 # Set threshold
cfg_dc5.DATASETS.TEST = ("robot_test",) # Give testset
predictor_dc5 = DefaultPredictor(cfg_dc5) #Make predictions

#### Evaluation matrix

In [ ]:
evaluator_dc5 = COCOEvaluator("robot_test", output_dir="./output") #Use coco evaluator
val_loader_dc5 = build_detection_test_loader(cfg_dc5, "robot_test")
print(inference_on_dataset(predictor_dc5.model, val_loader_dc5, evaluator_dc5)) #Print evaluation matrix

#### Visualization of predictions

In [ ]:
# Shows five random predictions from the test set
for d in random.sample(test_dicts, 5):       
    im = cv2.imread(d["file_name"])
    outputs = predictor_dc5(im) 
    v = Visualizer(im[:, :, ::-1],
                   metadata=microcontroller_metadata, 
                   scale=0.5
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize=(14, 10))
    plt.imshow(out.get_image()[:, :, ::-1][..., ::-1])

### Save model

In [ ]:
torch.save(cfg_dc5, 'model_dc5_final.pth')

## FPN backbone

### Train on baseline model

In [ ]:
cfg_fpn = get_cfg()
cfg_fpn.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml")) # Load model
cfg_fpn.DATASETS.TRAIN = ('robot_train',)
cfg_fpn.DATASETS.TEST = ('robot_valid',)  
cfg_fpn.DATALOADER.NUM_WORKERS = 2
cfg_fpn.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml") #Load weights
cfg_fpn.SOLVER.IMS_PER_BATCH = 4 #Set to four else it cannot run on collab due to resource limitations
cfg_fpn.SOLVER.MAX_ITER = 1000
cfg_fpn.SOLVER.STEPS = ()
cfg_fpn.MODEL.ROI_HEADS.NUM_CLASSES = 5 # Number of classes 

os.makedirs(cfg_fpn.OUTPUT_DIR, exist_ok=True) #Make directory for output of model
trainer_fpn = DefaultTrainer(cfg_fpn) # Use default trainer
trainer_fpn.resume_or_load(resume=False)
trainer_fpn.train() #Train model

#### Tensorboard over training

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

### Test and evaluation

In [ ]:
cfg_fpn.MODEL.WEIGHTS = os.path.join(cfg_fpn.OUTPUT_DIR, "model_fpn_final.pth")
cfg_fpn.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8 #Set threshold 
cfg_fpn.DATASETS.TEST = ("robot_test",) #Fed testset
predictor_fpn = DefaultPredictor(cfg_fpn) #Predict on testset

#### Evaluation matrix

In [ ]:
evaluator_fpn = COCOEvaluator("robot_test", output_dir="./output") # Use COCO evaluator
val_loader_fpn = build_detection_test_loader(cfg_fpn, "robot_test")
print(inference_on_dataset(predictor_fpn.model, val_loader_fpn, evaluator_fpn)) #Print evaluation

#### Visualization of predictions

In [ ]:
for d in random.sample(test_dicts, 5):       
    im = cv2.imread(d["file_name"])
    outputs = predictor_fpn(im) 
    v = Visualizer(im[:, :, ::-1],
                   metadata=microcontroller_metadata, 
                   scale=0.5
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize=(14, 10))
    plt.imshow(out.get_image()[:, :, ::-1][..., ::-1])

### Save model

In [ ]:
torch.save(cfg_dc5, 'model_dc5_final.pth')